In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from os.path import join as pjoin
sys.path.append('../')

from gensim.models import Word2Vec, KeyedVectors

from tst.io import AUTHORS, translation_embedding_dir
from tst.preprocess.parsing import find_anker_words
from tst.preprocess.markov import pos_emission_prob, pos_markov_chain, vocabulary, beam_search, \
    load_emission_probs, load_chain, load_pos_chain, beam_search2
from tst.preprocess.translate import translate_to_author, translate
from tst.preprocess.w2v_extensions import *
from tst.preprocess.helper import W2V
from tst.evaluation.evaluate import EvaluationBench

c:\users\kevin\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


## Create

In [ ]:
chains, pos_chains, emission_probs = {}, {}, {}
for author in os.listdir(AUTHORS)[2:]:
    chains[author], pos_chains[author] = pos_markov_chain(author, state_size=3)
    emission_probs[author] = pos_emission_prob(author)
    #g = markov_to_graph(pos_chain)

In [ ]:
dicts = {}
for author in os.listdir(AUTHORS):
    if author == 'all':
        wv = Word2Vec.load(pjoin(AUTHORS, author, 'parsed', 'gutenberg_w2v_5e.model')).wv
        vocab = {k: v.count for k, v in wv.vocab.items()}
    else:
        vocab = vocabulary(chains[author])
#         wv = KeyedVectors.load_word2vec_format(pjoin(AUTHORS, author, 'parsed', 'only.vec'))
#         vocab = {k: v.count for k, v in wv.vocab.items()}
        
    dicts[author] = normalize_dict(sorted(vocab, key=vocab.get, reverse=True))
    with open(pjoin(AUTHORS, author, 'parsed', 'dict.txt'), 'w') as f:
        f.writelines(map(lambda x: '{} {}\n'.format(x[0], x[1]), dicts[author].items()))

In [ ]:
for author in os.listdir(AUTHORS):
    if author == 'all':
        continue
        
    with open(pjoin(AUTHORS, author, 'parsed', 'ankers.txt'), 'w') as f:
        anker_words = find_anker_words(dicts['all'], dicts[author])
        for word in anker_words.keys():
            f.write(f'{word} {word}\n')

## Load

In [36]:
chains, pos_chains, emission_probs, embeddings = {}, {}, {}, {}
for author in os.listdir(AUTHORS):
    if author == 'all':
        continue
    
    chains[author] = load_chain(author)
    pos_chains[author] = load_pos_chain(author)
    emission_probs[author] = load_emission_probs(author)
    
    folder = translation_embedding_dir(author)
    embeddings[author] = [KeyedVectors.load_word2vec_format(pjoin(folder, 'vectors-all.txt')),
                          KeyedVectors.load_word2vec_format(pjoin(folder, f'vectors-{author}.txt'))]

## Evaluate

In [ ]:
['it', 'is', 'a', 'truth', 'generally', 'maintained', 'that', 'a',
       'single', 'man', 'in', 'prestige', 'of', 'a', 'good', 'intention',
       'will', 'be', 'in', 'want', 'of', 'a', 'daughter', '.'],

In [6]:
author = 'churchill'
# input_sent  = 'This is a quick test.'
input_sent = 'It is a truth universally acknowledged that a single man in possession of a good fortune must be in want of a wife.'
# words = translate(input_sent, embeddings[author][0], embeddings[author][1], .3)[0]
# words